In [1]:
import json
from pathlib import Path
import re
import sqlite3

_DB = 'college_dev.db'

conn = sqlite3.Connection(_DB)


In [52]:
# Build database
def init_db(script) -> None:
    "Initialize table using schema.sql and course_prefix.sql."
    with open(script, mode='r') as f:
        conn.executescript(f.read())
    conn.commit()
    return None

# Add course prefixes and create course_catalog table
init_db('sql_scripts_dev/course_catalog_schema.sql')
init_db('sql_scripts_dev/mycourses_schema.sql')

In [6]:
class Course:
    def __init__(self, prefix=None, number=None):
        """A college course."""
        self._prefix = prefix
        self._number = number
        self._title = None
        self._credits = None
        self._prereq = None
        self._description = None
    
    @property
    def prefix(self):
        return self._prefix

    @prefix.setter
    def prefix(self, value):
        self._prefix = value

    @property
    def number(self):
        return self._number

    @number.setter
    def number(self, value):
        self._number = value

    @property
    def title(self):
        return self._title

    @title.setter
    def title(self, value):
        self._title = value

    @property
    def credits(self):
        return self._credits

    @credits.setter
    def credits(self, value):
        self._credits = value

    @property
    def prereq(self):
        return self._prereq

    @prereq.setter
    def prereq(self, value):
        self._prereq = value

    @property
    def description(self):
        return self._description

    @description.setter
    def description(self, value):
        self._description = value

    @property
    def catalog_code(self):
        return f'{self.prefix}-{self.number}'

    def __repr__(self):
        return f'Course({self.prefix}, {self.number})'

    def __str__(self):
        return f'{self.catalog_code} {self.title}'


In [7]:
# Insert course catalog data into table from json files.
class CatalogPage:
    # Regex patterns to get course data.
    # Course catalog code can be AAA888, AA888, AA888A, or AA888A
    p_catalog_code = re.compile(r'^(\w?\w\w)(\d\d\d\w?)$')
    # Credits can be # CREDIT, #, 1 CREDIT, TITLE #, but not (#.
    p_credits = re.compile(r'(.*)?(?<![\(])(\d)( CREDIT[S]?)?')
    # First line of prerequisites always starts with PREREQUISITES:
    p_prereq = re.compile(r'[ ]?(PREREQUISITES: )(.*)')
    # The description is the only thing not in all capital letters.
    p_desc_start = re.compile(r'^([A-Z][a-z])')

    def __init__(self, page_path: Path) -> None:
        """Obtain the courses from a course catalog page."""
        # Open page file and read into mememory
        with open(page_path, 'r') as f:
            self.page: dict = json.loads(f.read().strip())
        # Pull out all text data from the page
        self._text = (span['text'].strip() for block in self.page['blocks']
                      for line in block['lines'] for span in line['spans'])
        self.pagename = next(self._text)
        self.source = next(self._text)
        next(self._text)  # Skip blank space
        self.pagenum = next(self._text)
        # To hold all the course data
        self.courses = []
        # Fill the courses list
        self._get_course_data()

    @property
    def _get_next_text(self) -> str:
        """Get next line of text from self._text_gen."""
        return next(self._text, None)

    def _parse_course_data(self, course_data, course):
        """Add course data to proper field in its Courses class."""
        title = []  # Course title
        credits = 0  # Number of credit hours for course
        pre_reqs = []  # Course pre-requisites
        description = []  # Course description

        for data in course_data:
            # Collect lines containing title until credit hours are found.
            if not credits:
                credit_match = self.p_credits.match(data)
                # Test for match with regex pattern for credits.
                if credit_match:
                    credits = int(credit_match.group(2))
                    # Sometimes credits is in the same line as the title.
                    if len(credit_match.group(1)) > 1:
                        title.append(credit_match.group(1))
                else:
                    title.append(data)
            # Build pre-requirements field until completed.  When
            # pre-requirements are collected, remaining items are the
            # description.
            elif not pre_reqs:
                pre_reqs_match = self.p_prereq.match(data)
                if pre_reqs_match:
                    pre_reqs.append(pre_reqs_match.group(2))
            elif not description:
                desc_match = self.p_desc_start.match(data)
                if not desc_match:
                    pre_reqs.append(data)
                else:
                    description.append(data)
            else:
                description.append(data)
        # Add details to finished course
        course.title = ' '.join(title).replace('*', '')
        course.credits = credits
        course.prereq = ' '.join(pre_reqs)
        course.description = ' '.join(description)

    def _get_course_data(self) -> None:
        """Get course data from new course up to description."""
        # Variables used to terminate loops
        num_courses = 0
        done = False
        # Loop over data until all course data obtained.
        while not done:
            # To hold course data before parsing.
            course_data = []
            # Collect course data until reaching the next course code.
            while num_courses == len(self.courses):
                text = self._get_next_text
                # Check if text generator is exhausted.
                if text is None:
                    # Change done to True and break to finish final course.
                    if self.courses[-1].title is None:
                        done = True
                        break
                # Skip two lines that contain a warning and not course data.
                if text.startswith('*'):
                    text = self._get_next_text
                    continue
                # If text is a catalog course code, register it and
                # terminate loop.
                cat_code_match = self.p_catalog_code.match(text)
                if cat_code_match:
                    # Add new course to courses list.
                    self.courses.append(Course(cat_code_match.group(1),
                                        cat_code_match.group(2)))
                    if num_courses == 0:
                        num_courses += 1
                        continue
                # Otherwise, continue collecting course data.
                else:
                    course_data.append(text)

            # Increase num_courses for next loop
            num_courses += 1
            # Parse collected course data and add to courses list.
            finished_course = self.courses[num_courses-2]
            self._parse_course_data(course_data, finished_course)


In [8]:
all_catalog_pages = []

def get_all_courses():
    files = tuple(Path('course_catalog').iterdir())
    for file in files:
        catalog_page = CatalogPage(file)
        all_catalog_pages.extend(catalog_page.courses)

get_all_courses()

In [ ]:
def add_courses_to_database():
    "Add all course catalog pages to database."


    def _get_db() -> sqlite3.Connection:
        """Create database connection."""
        conn = sqlite3.connect(
            _DB,
            detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES)
        conn.row_factory = sqlite3.Row
        return conn

    conn = _get_db()
    cur = conn.cursor()

    for course in all_catalog_pages:
        course_prefix = course.prefix
        cur.execute('SELECT rowid FROM course_prefixes WHERE prefix=?;',
                    [course_prefix])
        prefix_id = cur.fetchone()[0]
        if prefix_id is None:
            conn.commit()
            conn.close
            print(f'No prefix id match for {course_prefix}')
            return None
        cur.execute('''INSERT OR IGNORE INTO course_catalog(
                            course_prefix_id, course_number,
                            title, credits, prereq, description
                        )
                        VALUES(?, ?, ?, ?, ?, ?)''', [prefix_id, course.number, course.title, course.credits, course.prereq, course.description])
        conn.commit()
    conn.close()

add_courses_to_database()


In [44]:
# We can use the Field class to define models that describe the schema for each table in a database:

class Field:

    def __set_name__(self, owner, name):
        self.fetch = f'SELECT {name} FROM {owner.table} WHERE {owner.key}=?;'
        self.store = f'UPDATE {owner.table} SET {name}=? WHERE {owner.key}=?;'

    def __get__(self, obj, objtype=None):
        return conn.execute(self.fetch, [obj.key]).fetchone()[0]

    def __set__(self, obj, value):
        conn.execute(self.store, [value, obj.key])
        conn.commit()


In [47]:
# Course Catalog Tables for Querying

class CoursePrefix:
    table = 'course_prefixes'
    key = 'prefix'
    subject = Field()
    rowid = Field()

    def __init__(self, key):
        self.key = key

class CoursePrefixByID:
    table = 'course_prefixes'
    key = 'rowid'
    subject = Field()
    prefix = Field()

    def __init__(self, key):
        self.key = key

class CourseCatalog:
    table = 'course_catalog'
    key = 'uid'
    course_prefix_id = Field()
    course_number = Field()
    title = Field()
    credits = Field()
    pre_req = Field()
    description = Field()
    rowid = Field()
    pattern = re.compile(r'^([A-Za-z]{2,3})(-)?([0-9]{3}[A-Za-z]?)$')

    def __init__(self, key):
        match = self.pattern.match(key)
        if match:
            key = f'{CoursePrefix(match.group(1)).rowid}-{match.group(3)}'
        self.key = key

    @property
    def prefix(self):
        return CoursePrefixByID(self.course_prefix_id).prefix

    def __repr__(self):
        return f"CourseCatalog('{self.course_prefix_id}-{self.course_number}')"

    def __str__(self):
        return f'{self.prefix}-{self.course_number}: {self.title}'


In [45]:
# Personal Courses Tables for Querying

class Term:
    table = 'terms'
    key = 'rowid'
    start_date = Field()

    def __init__(self, key):
        self.key = key

class TermByDate:
    table = 'terms'
    key = 'start_date'
    num = Field()

    def __init__(self, key):
        self.key = key

class CourseTaken:
    table = 'courses_taken'
    key = 'uid'
    term_id = Field()
    course_catalog_id = Field()
    grade = Field()

    def __init__(self, key):
        self.key = key
        self.course_code = CourseCatalog(self.course_catalog_id)
        self.term_start = Term(self.term_id)

In [48]:
cs = CoursePrefix('CS')
cs.rowid

cs197 = CourseCatalog('11-197')
cs197.title
cs197.description
cs197.course_prefix_id
cs197.rowid

407

In [ ]:
conn.close()